<a href="https://colab.research.google.com/github/abdul9870/abdul9870/blob/main/project_2_Prompt_Engineering_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Hands-On Prompt Engineering with LLMs

## Introduction and Objectives
This notebook provides hands-on practice with various prompt engineering techniques.
We will interact with a Large Language Model (LLM) to perform tasks like classification, Q&A, and simple reasoning.
By the end of this session, you will be able to:
- Understand and apply zero-shot, few-shot, and Chain-of-Thought (CoT) prompting.
- Create and iterate on prompts for specific tasks.
- Compare the effectiveness of different prompting strategies.
- Gain experience interacting with LLMs.

## 1. Setup - Install and Import Libraries

# Ensure necessary libraries are installed. Uncomment and run this cell in your environment if needed.

In [ ]:
# We will primarily use Hugging Face Transformers for accessing an LLM.
!pip install transformers torch accelerate sentencepiece -q # sentencepiece is often needed for T5 models

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 16.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.4 MB/s eta 0:00:00
ERROR: Operation cancelled by user


In [ ]:
# Import required libraries
import torch
from transformers import pipeline

## 2. Choosing and Accessing an LLM
For this notebook, we will use a relatively small and accessible text-to-text generation model
from the Hugging Face Hub, such as a FLAN-T5 variant. This allows us to experiment
without needing API keys for proprietary models. If you have access to other models
(e.g., via OpenAI API), you can adapt the `get_llm_response` function accordingly.

**Important Note on API Keys (if using API-based models like OpenAI):**
If you adapt this notebook to use an API that requires keys (e.g., OpenAI GPT models):
1.  You will need your own API key.
2.  **NEVER hardcode your API key directly in the notebook.**
3.  Store it securely, for example, as an environment variable or use a secure input method if your environment supports it.
    - In Colab, you can use the "Secrets" manager.
    - Locally, you can set environment variables (e.g., `OPENAI_API_KEY`).

In [ ]:
# Load a text2text-generation pipeline with a FLAN-T5 model
# Using a smaller model for faster inference in a teaching setting.
# You can try other models like "google/flan-t5-base" or "google/flan-t5-large" for better performance (but slower).
MODEL_CHOICE = "google/flan-t5-small"

try:
    print(f"Loading LLM pipeline with model: {MODEL_CHOICE}...")
    llm_pipeline = pipeline("text2text-generation", model=MODEL_CHOICE, device=0 if torch.cuda.is_available() else -1)
    print("LLM pipeline loaded successfully.")
except Exception as e:
    print(f"Error loading LLM pipeline: {e}")
    print("Please ensure you have a working internet connection and the necessary libraries installed.")
    llm_pipeline = None # Set to None if loading fails

# Helper function to get LLM response (works with the Hugging Face pipeline)
def get_llm_response(prompt_text, max_length=150):
    if llm_pipeline is None:
        return "Error: LLM pipeline not loaded."
    try:
        # The pipeline returns a list of dictionaries
        response = llm_pipeline(prompt_text, max_length=max_length)
        return response[0]["generated_text"].strip()
    except Exception as e:
        return f"Error during LLM inference: {e}"

Loading LLM pipeline with model: google/flan-t5-small...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

...

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Device set to use cpu


LLM pipeline loaded successfully.


In [ ]:
# Test the LLM pipeline with a simple prompt
if llm_pipeline:
    test_prompt = "Translate to German: Hello, how are you?"
    test_response = get_llm_response(test_prompt)
    print(f"Test Prompt: {test_prompt}")
    print(f"Test Response: {test_response}")
else:
    print("Skipping LLM test as pipeline failed to load.")

Test Prompt: Translate to German: Hello, how are you?
Test Response: Hello, wie ich ich?


## 3. Task 1 - Zero-Shot Prompting
Zero-shot prompting involves asking the LLM to perform a task without providing any prior examples of that specific task in the prompt. The LLM relies on its pre-trained knowledge.

### 3.1 Sentiment Classification (Zero-Shot)

In [ ]:
review1 = "This movie was absolutely fantastic! The acting was superb, and the plot was engaging. Highly recommended."
review2 = "I was really disappointed with this film. It was boring, predictable, and far too long."

prompt_sentiment_zero_shot_template = "Classify the sentiment of the following movie review as positive, negative, or neutral. Review: {review} Sentiment:"

if llm_pipeline:
    prompt1 = prompt_sentiment_zero_shot_template.format(review=review1)
    response1 = get_llm_response(prompt1)
    print(f"Review: {review1}")
    # print(f"Prompt: {prompt1}") # You can uncomment to see the full prompt
    print(f"LLM Output (Sentiment): {response1}")

    print("---")
    prompt2 = prompt_sentiment_zero_shot_template.format(review=review2)
    response2 = get_llm_response(prompt2)
    print(f"Review: {review2}")
    # print(f"Prompt: {prompt2}")
    print(f"LLM Output (Sentiment): {response2}")
else:
    print("Skipping Zero-Shot Sentiment Classification as LLM pipeline is not loaded.")

Review: This movie was absolutely fantastic! The acting was superb, and the plot was engaging. Highly recommended.
LLM Output (Sentiment): positive
---
Review: I was really disappointed with this film. It was boring, predictable, and far too long.
LLM Output (Sentiment): negative


### 3.2 Summarization (Zero-Shot)

In [ ]:
paragraph_to_summarize = "The James Webb Space Telescope (JWST) is a space telescope designed primarily to conduct infrared astronomy. As the largest optical telescope in space, its high infrared resolution and sensitivity allow it to view objects too old, distant, or faint for the Hubble Space Telescope. This is expected to enable a broad range of investigations across the fields of astronomy and cosmology, such as observation of the first stars and the formation of the first galaxies, and detailed atmospheric characterization of potentially habitable exoplanets."
prompt_summarize_zero_shot = f"Summarize the following paragraph in one concise sentence: \nParagraph: {paragraph_to_summarize}\nSummary:"

if llm_pipeline:
    summary_response = get_llm_response(prompt_summarize_zero_shot, max_length=50)
    print(f"Original Paragraph: {paragraph_to_summarize}")
    print(f"LLM Summary: {summary_response}")
else:
    print("Skipping Zero-Shot Summarization as LLM pipeline is not loaded.")

Original Paragraph: The James Webb Space Telescope (JWST) is a space telescope designed primarily to conduct infrared astronomy. As the largest optical telescope in space, its high infrared resolution and sensitivity allow it to view objects too old, distant, or faint for the Hubble Space Telescope. This is expected to enable a broad range of investigations across the fields of astronomy and cosmology, such as observation of the first stars and the formation of the first galaxies, and detailed atmospheric characterization of potentially habitable exoplanets.
LLM Summary: The James Webb Space Telescope is a space telescope designed to conduct infrared astronomy.


## 4. Task 2 - Few-Shot Prompting
Few-shot prompting involves providing a few examples (input-output pairs, or "shots") of the task within the prompt to guide the LLM. This helps the model understand the task and desired output format better through in-context learning.

### 4.1 Sentiment Classification (Few-Shot)
Let's try to improve sentiment classification by providing examples.

In [ ]:
review_new = "The movie was okay, not the best I've seen but had some good moments."

prompt_sentiment_few_shot = f"""Classify the sentiment of the following movie reviews as positive, negative, or neutral.

Review: I loved this movie! It was amazing.
Sentiment: positive

Review: This film was a complete waste of time.
Sentiment: negative

Review: It was an average movie, nothing special.
Sentiment: neutral

Review: {review_new}
Sentiment:"""

if llm_pipeline:
    few_shot_response = get_llm_response(prompt_sentiment_few_shot)
    print(f"New Review: {review_new}")
    # print(f"Few-Shot Prompt: \n{prompt_sentiment_few_shot}") # Uncomment to see the full prompt
    print(f"LLM Output (Sentiment, Few-Shot): {few_shot_response}")
    # Compare this with a zero-shot attempt for `review_new` if you like.
else:
    print("Skipping Few-Shot Sentiment Classification as LLM pipeline is not loaded.")

New Review: The movie was okay, not the best I've seen but had some good moments.
LLM Output (Sentiment, Few-Shot): positive


### 4.2 Custom Task: Extracting Product Features (Few-Shot)

In [ ]:
product_description_new = "Introducing the new SuperPhone X, featuring a stunning 6.7-inch OLED display, an advanced A20 Bionic chip, and a triple-camera system with 8K video recording."

prompt_extract_features_few_shot = f"""Extract key features from the product descriptions.

Description: The SmartBook Pro laptop comes with a 13-inch Retina display, M2 chip, and 16GB RAM.
Features: 13-inch Retina display, M2 chip, 16GB RAM

Description: Our new SoundWave headphones offer active noise cancellation, 30-hour battery life, and Bluetooth 5.2.
Features: active noise cancellation, 30-hour battery life, Bluetooth 5.2

Description: {product_description_new}
Features:"""

if llm_pipeline:
    features_response = get_llm_response(prompt_extract_features_few_shot, max_length=100)
    print(f"New Product Description: {product_description_new}")
    print(f"LLM Output (Extracted Features, Few-Shot): {features_response}")
else:
    print("Skipping Few-Shot Feature Extraction as LLM pipeline is not loaded.")

New Product Description: Introducing the new SuperPhone X, featuring a stunning 6.7-inch OLED display, an advanced A20 Bionic chip, and a triple-camera system with 8K video recording.
LLM Output (Extracted Features, Few-Shot): SuperPhone X features a stunning 6.7-inch OLED display, an advanced A20 Bionic chip, and a triple-camera system with 8K video recording.


## 5. Task 3 - Chain-of-Thought (CoT) Prompting
Chain-of-Thought prompting encourages the LLM to generate a series of intermediate reasoning steps before giving the final answer. This is particularly useful for tasks requiring complex reasoning, like math word problems or logical deductions.
We often elicit CoT by including "Let's think step by step" or similar phrases in our few-shot examples or directly in the prompt for the new problem.

### 5.1 Math Word Problem (CoT)

In [ ]:
word_problem_cot_example = "Q: Roger has 5 tennis balls. He buys 2 more cans of tennis balls. Each can has 3 tennis balls. How many tennis balls does he have now?\nA: Let's think step by step. Roger starts with 5 balls. He buys 2 cans, and each can has 3 balls. So, he gets 2 * 3 = 6 more balls. In total, he has 5 + 6 = 11 balls. The final answer is 11."

new_word_problem = "Natalia sold clips to 4 of her friends. If she sold 8 clips to each friend, how many clips did she sell in total?"

prompt_solve_cot = f"{word_problem_cot_example}\n\nQ: {new_word_problem}\nA: Let's think step by step."

if llm_pipeline:
    cot_math_response = get_llm_response(prompt_solve_cot, max_length=200)
    print(f"New Word Problem: {new_word_problem}")
    print(f"LLM Solution (CoT): {cot_math_response}")

    # For comparison, a non-CoT prompt (might be less reliable for complex problems)
    prompt_solve_direct = f"Q: {new_word_problem}\nA:"
    direct_math_response = get_llm_response(prompt_solve_direct, max_length=50)
    print(f"LLM Solution (Direct): {direct_math_response}")
else:
    print("Skipping CoT Math Problem as LLM pipeline is not loaded.")

New Word Problem: Natalia sold clips to 4 of her friends. If she sold 8 clips to each friend, how many clips did she sell in total?
LLM Solution (CoT): Natalia sold 8 * 4 = 24 clips. So, Natalia sold 24 + 8 = 66 clips. So, Natalia sold 66 + 66 = 66 clips. The final answer is 66.
LLM Solution (Direct): 8


## 6. Task 4 - Iterating and Comparing Custom Prompts
Prompt engineering is an iterative process. For a given task, you might try several different prompts to see which yields the best results.
Key elements to vary include: clarity, specificity, amount of context, constraints, and desired output format.

### 6.1 Generating Taglines
Goal: Generate a catchy tagline for a new eco-friendly coffee shop.

In [ ]:
goal_tagline = "Generate a catchy tagline for a new eco-friendly coffee shop called 'GreenSprout Cafe'."

# Prompt V1: Very simple
prompt_tagline_v1 = f"Tagline for GreenSprout Cafe, an eco-friendly coffee shop:"

# Prompt V2: More specific about desired output
prompt_tagline_v2 = f"Write a short, catchy, and memorable tagline (under 10 words) for a new coffee shop named 'GreenSprout Cafe'. The shop emphasizes its commitment to sustainability, organic ingredients, and a cozy atmosphere."

# Prompt V3: Using role-playing and asking for multiple options
prompt_tagline_v3 = f"Act as a creative marketing expert. Generate three distinct taglines for 'GreenSprout Cafe'. This coffee shop is known for its fair-trade organic coffee, locally sourced pastries, and strong focus on environmental sustainability. Taglines should be inspiring and highlight its green values."

if llm_pipeline:
    print(f"Goal: {goal_tagline}")
    print("---")

    response_tagline_v1 = get_llm_response(prompt_tagline_v1, max_length=50)
    print(f"Prompt V1: {prompt_tagline_v1}")
    print(f"LLM Output V1: {response_tagline_v1}")
    print("---")

    response_tagline_v2 = get_llm_response(prompt_tagline_v2, max_length=50)
    print(f"Prompt V2: {prompt_tagline_v2}")
    print(f"LLM Output V2: {response_tagline_v2}")
    print("---")

    response_tagline_v3 = get_llm_response(prompt_tagline_v3, max_length=100)
    print(f"Prompt V3: {prompt_tagline_v3}")
    print(f"LLM Output V3: {response_tagline_v3}")
else:
    print("Skipping Custom Prompt Comparison as LLM pipeline is not loaded.")

Goal: Generate a catchy tagline for a new eco-friendly coffee shop called 'GreenSprout Cafe'.
---
Prompt V1: Tagline for GreenSprout Cafe, an eco-friendly coffee shop:
LLM Output V1: GreenSprout Cafe is a coffee shop that is eco friendly and serves food.
---
Prompt V2: Write a short, catchy, and memorable tagline (under 10 words) for a new coffee shop named 'GreenSprout Cafe'. The shop emphasizes its commitment to sustainability, organic ingredients, and a cozy atmosphere.
LLM Output V2: GreenSprout Cafe is a coffee shop that is a little bit more than a coffee shop.
---
Prompt V3: Act as a creative marketing expert. Generate three distinct taglines for 'GreenSprout Cafe'. This coffee shop is known for its fair-trade organic coffee, locally sourced pastries, and strong focus on environmental sustainability. Taglines should be inspiring and highlight its green values.
LLM Output V3: GreenSprout Cafe


## 7. Conclusion and Further Exploration
In this notebook, we explored several fundamental prompt engineering techniques:
- **Zero-Shot Prompting:** Leveraging the LLM's existing knowledge without examples.
- **Few-Shot Prompting:** Guiding the LLM with a few in-context examples.
- **Chain-of-Thought (CoT) Prompting:** Encouraging step-by-step reasoning for complex tasks.
- **Iterative Prompt Design:** Experimenting with different phrasings, context, and constraints.

Effective prompt engineering is key to unlocking the full potential of LLMs. It often involves creativity, experimentation, and a good understanding of the LLM's capabilities and limitations.

**Further Exploration Ideas:**
- Experiment with different LLMs (e.g., other models from Hugging Face Hub).
- Try more complex tasks requiring multi-turn conversations or more elaborate CoT.
- Explore prompt templating libraries if you build applications.
- Investigate the impact of generation parameters like `temperature`, `top_p`, `top_k` (if your pipeline/API supports them).

## 8. Next Steps
In the next session (Day 4 of the course), we will look into building a Text Generation CLI App using LangChain, which often involves sophisticated prompt management.

print("\nPrompt Engineering Notebook script finished.")